# Final Project - Reinforcements Learning 
Hello dear students,<br> this is the template notebook. Please click on the "File" tab and then on "Save a copy into drive".

---
<br>

### Name and ID:
Student 1: Avraham Raviv, 204355390
<br>
Student 2: Yevgeni Berkovitch, 317079234
<br><br>
<img src="https://play-lh.googleusercontent.com/e_oKlKPISbgdzut1H9opevS7-LTB8-8lsmpCdMkhlnqFenZhpjxbLmx7l158-xQQCIY">

### https://github.com/mpSchrader/gym-sokoban

# Installs

In [1]:
%%capture
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install gym
!pip install pygame
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install gym_sokoban

!imageio_download_bin ffmpeg

# Imports

In [2]:
import random
import time

import numpy as np
import scipy as scp
import matplotlib.pyplot as plt

import base64
import imageio
from pyvirtualdisplay import Display
from IPython.display import HTML

import gym
from gym import error, spaces, utils
from soko_pap import *

from collections import deque
from queue import PriorityQueue

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

from tqdm.notebook import tqdm
from collections import defaultdict

In [3]:
%matplotlib inline

In [4]:
imageio.plugins.ffmpeg.download()

In [5]:
from gym import logger as gymlogger
gymlogger.set_level(40) # error only

# Display utils
The cell below contains the video display configuration. No need to make changes here.

In [6]:
def embed_mp4(filename):
    """Embeds an mp4 file in the notebook."""
    video = open(filename,'rb').read()
    b64 = base64.b64encode(video)
    tag = '''
    <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
    Your browser does not support the video tag.
    </video>'''.format(b64.decode())

    return HTML(tag)

# Utils

In [7]:
def get_distances(room_state):
    for i in range(room_state.shape[0]):
        for j in range(room_state.shape[1]):
            if room_state[i][j] == 2:
                target = (i, j)

    distances = np.zeros(shape=room_state.shape)
    visited_cells = set()
    cell_queue = deque()

    visited_cells.add(target)
    cell_queue.appendleft(target)

    while len(cell_queue) != 0:
        cell = cell_queue.pop()
        distance = distances[cell[0]][cell[1]]
        for x,y in ((1,0), (-1,-0), (0,1), (0,-1)):
            next_cell_x, next_cell_y = cell[0]+x, cell[1]+y
            if room_state[next_cell_x][next_cell_y] != 0 and not (next_cell_x, next_cell_y) in visited_cells:
                distances[next_cell_x][next_cell_y] = distance + 1
                visited_cells.add((next_cell_x, next_cell_y))
                cell_queue.appendleft((next_cell_x, next_cell_y))
                
    return distances   

def calc_distances(room_state, distances):
    box = None
    mover = None
    for i in range(room_state.shape[0]):
        for j in range(room_state.shape[1]):            
            if room_state[i][j] == 4:
                box = (i,j)
            
            if room_state[i][j] == 5:
                mover = (i,j)
    
    return mover, box, distances[box[0]][box[1]]   

def box2target_change_reward(room_state, next_room_state, distances):
    if np.array_equal(room_state, next_room_state):
        return -1.0
    
    mover, box, t2b = calc_distances(room_state, distances)
    n_mover, n_box, n_t2b = calc_distances(next_room_state, distances)
    
    change_reward = 0.0
    if n_t2b < t2b:
        change_reward += 5.0
    elif n_t2b > t2b:
        change_reward -= 5.0
        
    m2b = np.sqrt((mover[0]-box[0])**2 + (mover[1]-box[1])**2)
    n_m2b = np.sqrt((n_mover[0]-n_box[0])**2 + (n_mover[1]-n_box[1])**2)
    
    if n_m2b < m2b and m2b >= 2:
        change_reward += 1.0
    elif n_m2b > m2b and n_m2b >= 2:
        change_reward -= 1.0
        
    return change_reward   

# Solution

In [8]:
class SOK_Agent:
    def __init__(self):
        # Construct DQN models
        self.state_size = (7,7,1) 
        self.action_size = 8
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.target_model.set_weights(self.model.get_weights())
        self.batch_size = 8
        
        # Replay buffers
        self.replay_buffer = deque(maxlen=5000)
        self.prioritized_replay_buffer = deque(maxlen=500)
        
        # Hyperparameters
        self.gamma = 0.9
        self.epsilon = 1.0   
        self.epsilon_min = 0.3
        self.epsilon_decay = 0.995
        self.replay_rate = 10
        self.update_beta = 0.999

        self.verbosity = 100 

    def _build_model(self):
        model = Sequential()
        model.add(Conv2D(64, (3, 3), input_shape=self.state_size, activation='relu'))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))       
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer="adam")        
        return model

    def remember(self, state, action, reward, next_state, done):
        self.replay_buffer.append([state, action, reward, next_state, done])    
        
    def copy_to_prioritized_buffer(self, n):
        for i in range(n):
            self.prioritized_replay_buffer.append(self.replay_buffer[-1-i])  

    def act(self, state, stochastic=False):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        act_values = self.model.predict(state, verbose=0)[0]
        
        if stochastic:
            act_probs = np.exp(act_values)/np.exp(act_values).sum()
            return np.random.choice(np.arange(self.action_size), size=1, p=act_probs)[0]
              
        return np.argmax(act_values) 

    def replay(self): 
        if len(self.replay_buffer) < self.batch_size:
            return
        
        if len(self.prioritized_replay_buffer) < self.batch_size//2:
            minibatch = random.sample(self.replay_buffer, self.batch_size) 
        else:    
            minibatch = random.sample(self.replay_buffer, self.batch_size//2) 
            minibatch.extend(random.sample(self.prioritized_replay_buffer, self.batch_size//2))
        
        states = np.zeros((self.batch_size, self.state_size[0], self.state_size[1]))
        actions = np.zeros(self.batch_size, dtype=int)
        rewards = np.zeros(self.batch_size)
        next_states = np.zeros((self.batch_size, self.state_size[0], self.state_size[1]))
        statuses = np.zeros(self.batch_size)
        targets = np.zeros((self.batch_size, self.action_size)) 
        
        for i, (state, action, reward, next_state, done) in enumerate(minibatch): 
            states[i] = state.copy()
            actions[i] = action
            rewards[i] = reward
            next_states[i] = next_state.copy()
            statuses[i] = 1 if done else 0    
        
        targets = self.model.predict(states) 
        max_actions = np.argmax(self.model.predict(next_states), axis=1)
        next_rewards = self.target_model.predict(next_states)
        
        ind = 0
        for action, reward, next_reward, max_action, done in zip(actions, rewards, next_rewards, max_actions, statuses):  
            if not done:
                reward += self.gamma * next_reward[max_action]
            targets[ind][action] = reward
            ind += 1
        
        self.model.fit(states, targets, epochs=10, verbose=0) 
        
        self.update_target_model()        
    
        if self.epsilon > self.epsilon_min:
            self.epsilon = self.epsilon * self.epsilon_decay
        
    def update_target_model(self):
        model_w = self.model.get_weights()
        target_model_w = self.target_model.get_weights()
        updated_target_model_w = []
        for i in range(len(model_w)):
            updated_target_model_w.append(self.update_beta*target_model_w[i] + (1-self.update_beta)*model_w[i])
        self.target_model.set_weights(updated_target_model_w)    
            
    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [9]:
def process_frame(frame):
    f = frame[:, :, 0]   
    f = f.reshape(7, 16, 7, 16).max(axis=(1, 3))
    #f = f.flatten()
    f = f / 255
    return np.expand_dims(f, axis=0)

## Training

#### Test Suite

In [10]:
def test_agent(stochastic=False):
    current_epsilon = agent.epsilon
    agent.epsilon = 0.0
    num_solved = 0
    solved_in_steps = defaultdict(int)

    for t in tqdm(range(100)):    
        random.seed(t)
        sok = PushAndPullSokobanEnv(dim_room=(7, 7), num_boxes=1)
        sok.set_maxsteps(20)
        steps = 0

        state = sok.get_image('rgb_array')
        done = False
        while not done:
            steps += 1
            action = agent.act(process_frame(state), stochastic)
            if action < 4:
                action += 1
            else:
                action += 5
            state, reward, done, info = sok.step(action)

        if 3 in sok.room_state:            
            num_solved += 1
            solved_in_steps[steps] += 1
    
    agent.epsilon = current_epsilon
    print("*" * 30)
    print("Stochastic" if stochastic else "Deterministic")
    print("*" * 30)
    print("Solved: %d" % num_solved)
    print("=" * 30)
    print(solved_in_steps)
    print("*" * 30)

In [11]:
max_episodes = 50000
max_steps = 25

def init_sok(r):
    random.seed(r+100)
    sok = PushAndPullSokobanEnv(dim_room=(7, 7), num_boxes=1)
    sok.set_maxsteps(max_steps)
    return sok

In [ ]:
agent = SOK_Agent()

steps_per_episode = []

for e in range(max_episodes):
    if e % 100 == 0 and e > 0:
        test_agent(stochastic=False)
        
    print("Episode: %d" % (e))
    
    sok = init_sok(e)
    random.seed(e)
    
    state = process_frame(sok.get_image('rgb_array'))
    room_state = sok.room_state.copy() 
    distances = get_distances(room_state)
    
    for step in range(sok.max_steps):
        action = agent.act(state)
        if action < 4:
            next_state, reward, done, _ = sok.step(action+1) 
        else:
            next_state, reward, done, _ = sok.step(action+5)         
        
        next_state = process_frame(next_state)        
        next_room_state = sok.room_state
        
        if not done:
            reward += box2target_change_reward(room_state, next_room_state, distances)
        
        agent.remember(state, action, reward, next_state, done)
        
        state = next_state.copy() 
        room_state = next_room_state.copy()                
        
        if (step+1) % agent.replay_rate == 0:
            agent.replay()            
        
        if done:   
            steps_per_episode.append(step+1)
            
            if 3 in sok.room_state:                
                print("SOLVED! Episode %d Steps: %d Epsilon %.4f" % (e, step+1, agent.epsilon)) 
                agent.copy_to_prioritized_buffer(step+1)
            
            #agent.save("exp1_episode%d.h5" % (e))            
            break

Episode: 0
Episode: 1
Episode: 2
Episode: 3
Episode: 4
Episode: 5
Episode: 6
Episode: 7
SOLVED! Episode 7 Steps: 6 Epsilon 0.9322
Episode: 8
Episode: 9
Episode: 10
SOLVED! Episode 10 Steps: 12 Epsilon 0.9092
Episode: 11
Episode: 12
Episode: 13
SOLVED! Episode 13 Steps: 7 Epsilon 0.8911
Episode: 14
Episode: 15
Episode: 16
SOLVED! Episode 16 Steps: 19 Epsilon 0.8691
Episode: 17
Episode: 18
Episode: 19
Episode: 20
Episode: 21
Episode: 22
SOLVED! Episode 22 Steps: 13 Epsilon 0.8224
Episode: 23
Episode: 24
Episode: 25
Episode: 26
SOLVED! Episode 26 Steps: 4 Epsilon 0.7981
Episode: 27
Episode: 28
Episode: 29
Episode: 30
Episode: 31
Episode: 32
Episode: 33
Episode: 34
Episode: 35
Episode: 36
Episode: 37
Episode: 38
Episode: 39
Episode: 40
Episode: 41
SOLVED! Episode 41 Steps: 3 Epsilon 0.6936
Episode: 42
Episode: 43
Episode: 44
Episode: 45
Episode: 46
SOLVED! Episode 46 Steps: 3 Epsilon 0.6663
Episode: 47
SOLVED! Episode 47 Steps: 3 Epsilon 0.6663
Episode: 48
SOLVED! Episode 48 Steps: 1 Epsil


******************************
Deterministic
******************************
Solved: 24
defaultdict(<class 'int'>, {3: 9, 1: 10, 2: 5})
******************************
Episode: 100
Episode: 101
Episode: 102
SOLVED! Episode 102 Steps: 18 Epsilon 0.4244
Episode: 103
SOLVED! Episode 103 Steps: 5 Epsilon 0.4244
Episode: 104
Episode: 105
Episode: 106
Episode: 107
SOLVED! Episode 107 Steps: 3 Epsilon 0.4118
Episode: 108
Episode: 109
Episode: 110
Episode: 111
Episode: 112
Episode: 113
Episode: 114
SOLVED! Episode 114 Steps: 1 Epsilon 0.3878
Episode: 115
Episode: 116
Episode: 117
Episode: 118
SOLVED! Episode 118 Steps: 1 Epsilon 0.3763
Episode: 119
Episode: 120
Episode: 121
Episode: 122
Episode: 123
Episode: 124
SOLVED! Episode 124 Steps: 3 Epsilon 0.3579
Episode: 125
Episode: 126
Episode: 127
Episode: 128
SOLVED! Episode 128 Steps: 2 Epsilon 0.3473
Episode: 129
Episode: 130
Episode: 131
Episode: 132
Episode: 133
SOLVED! Episode 133 Steps: 23 Epsilon 0.3303
Episode: 134
Episode: 135
SOLVED! Epi


******************************
Deterministic
******************************
Solved: 28
defaultdict(<class 'int'>, {3: 13, 1: 9, 2: 6})
******************************
Episode: 200
SOLVED! Episode 200 Steps: 15 Epsilon 0.2988
Episode: 201
Episode: 202
Episode: 203
Episode: 204
Episode: 205
Episode: 206
Episode: 207
Episode: 208
Episode: 209
SOLVED! Episode 209 Steps: 9 Epsilon 0.2988
Episode: 210
Episode: 211
Episode: 212
Episode: 213
SOLVED! Episode 213 Steps: 3 Epsilon 0.2988
Episode: 214
SOLVED! Episode 214 Steps: 3 Epsilon 0.2988
Episode: 215
Episode: 216
SOLVED! Episode 216 Steps: 3 Epsilon 0.2988
Episode: 217
Episode: 218
SOLVED! Episode 218 Steps: 2 Epsilon 0.2988
Episode: 219
Episode: 220
SOLVED! Episode 220 Steps: 6 Epsilon 0.2988
Episode: 221
SOLVED! Episode 221 Steps: 1 Epsilon 0.2988
Episode: 222
Episode: 223
SOLVED! Episode 223 Steps: 1 Epsilon 0.2988
Episode: 224
Episode: 225
Episode: 226
Episode: 227
Episode: 228
Episode: 229
Episode: 230
SOLVED! Episode 230 Steps: 23 Eps


******************************
Deterministic
******************************
Solved: 30
defaultdict(<class 'int'>, {3: 16, 2: 4, 1: 10})
******************************
Episode: 300
SOLVED! Episode 300 Steps: 1 Epsilon 0.2988
Episode: 301
Episode: 302
Episode: 303
Episode: 304
Episode: 305
SOLVED! Episode 305 Steps: 2 Epsilon 0.2988
Episode: 306
Episode: 307
Episode: 308
Episode: 309
Episode: 310
Episode: 311
Episode: 312
Episode: 313
Episode: 314
Episode: 315
SOLVED! Episode 315 Steps: 3 Epsilon 0.2988
Episode: 316
Episode: 317
Episode: 318
Episode: 319
SOLVED! Episode 319 Steps: 3 Epsilon 0.2988
Episode: 320
Episode: 321
Episode: 322
SOLVED! Episode 322 Steps: 1 Epsilon 0.2988
Episode: 323
SOLVED! Episode 323 Steps: 3 Epsilon 0.2988
Episode: 324
Episode: 325
Episode: 326
Episode: 327
SOLVED! Episode 327 Steps: 2 Epsilon 0.2988
Episode: 328
Episode: 329
Episode: 330
SOLVED! Episode 330 Steps: 2 Epsilon 0.2988
Episode: 331
Episode: 332
Episode: 333
Episode: 334
SOLVED! Episode 334 Steps


******************************
Deterministic
******************************
Solved: 32
defaultdict(<class 'int'>, {3: 16, 2: 6, 1: 10})
******************************
Episode: 400
Episode: 401
Episode: 402
Episode: 403
Episode: 404
SOLVED! Episode 404 Steps: 3 Epsilon 0.2988
Episode: 405
SOLVED! Episode 405 Steps: 1 Epsilon 0.2988
Episode: 406
Episode: 407
Episode: 408
Episode: 409
Episode: 410
Episode: 411
Episode: 412
Episode: 413
SOLVED! Episode 413 Steps: 3 Epsilon 0.2988
Episode: 414
Episode: 415
Episode: 416
SOLVED! Episode 416 Steps: 3 Epsilon 0.2988
Episode: 417
Episode: 418
SOLVED! Episode 418 Steps: 3 Epsilon 0.2988
Episode: 419
Episode: 420
Episode: 421
Episode: 422
Episode: 423
Episode: 424
Episode: 425
Episode: 426
Episode: 427
SOLVED! Episode 427 Steps: 3 Epsilon 0.2988
Episode: 428
Episode: 429
SOLVED! Episode 429 Steps: 6 Epsilon 0.2988
Episode: 430
Episode: 431
Episode: 432
Episode: 433
Episode: 434
Episode: 435
SOLVED! Episode 435 Steps: 11 Epsilon 0.2988
Episode: 43

...


******************************
Deterministic
******************************
Solved: 42
defaultdict(<class 'int'>, {3: 20, 1: 15, 2: 7})
******************************
Episode: 3200
SOLVED! Episode 3200 Steps: 23 Epsilon 0.2988
Episode: 3201
Episode: 3202
SOLVED! Episode 3202 Steps: 1 Epsilon 0.2988
Episode: 3203
Episode: 3204
SOLVED! Episode 3204 Steps: 1 Epsilon 0.2988
Episode: 3205
Episode: 3206
Episode: 3207
SOLVED! Episode 3207 Steps: 3 Epsilon 0.2988
Episode: 3208
Episode: 3209
Episode: 3210
SOLVED! Episode 3210 Steps: 2 Epsilon 0.2988
Episode: 3211
Episode: 3212
Episode: 3213
SOLVED! Episode 3213 Steps: 3 Epsilon 0.2988
Episode: 3214
Episode: 3215
Episode: 3216
Episode: 3217
Episode: 3218
SOLVED! Episode 3218 Steps: 9 Epsilon 0.2988
Episode: 3219
SOLVED! Episode 3219 Steps: 1 Epsilon 0.2988
Episode: 3220
SOLVED! Episode 3220 Steps: 1 Epsilon 0.2988
Episode: 3221
Episode: 3222
Episode: 3223
SOLVED! Episode 3223 Steps: 1 Epsilon 0.2988
Episode: 3224
SOLVED! Episode 3224 Steps: 3 E


******************************
Deterministic
******************************
Solved: 36
defaultdict(<class 'int'>, {3: 18, 1: 11, 2: 7})
******************************
Episode: 3300
Episode: 3301
Episode: 3302
Episode: 3303
Episode: 3304
SOLVED! Episode 3304 Steps: 3 Epsilon 0.2988
Episode: 3305
SOLVED! Episode 3305 Steps: 3 Epsilon 0.2988
Episode: 3306
SOLVED! Episode 3306 Steps: 1 Epsilon 0.2988
Episode: 3307
SOLVED! Episode 3307 Steps: 13 Epsilon 0.2988
Episode: 3308
Episode: 3309
Episode: 3310
Episode: 3311
SOLVED! Episode 3311 Steps: 1 Epsilon 0.2988
Episode: 3312
Episode: 3313
Episode: 3314
Episode: 3315
Episode: 3316
Episode: 3317
Episode: 3318
Episode: 3319
SOLVED! Episode 3319 Steps: 2 Epsilon 0.2988
Episode: 3320
Episode: 3321
Episode: 3322
Episode: 3323
Episode: 3324
Episode: 3325
Episode: 3326
SOLVED! Episode 3326 Steps: 15 Epsilon 0.2988
Episode: 3327
Episode: 3328
SOLVED! Episode 3328 Steps: 1 Epsilon 0.2988
Episode: 3329
Episode: 3330
Episode: 3331
SOLVED! Episode 3331 S


******************************
Deterministic
******************************
Solved: 44
defaultdict(<class 'int'>, {3: 19, 1: 17, 2: 8})
******************************
Episode: 3400
Episode: 3401
Episode: 3402
SOLVED! Episode 3402 Steps: 1 Epsilon 0.2988
Episode: 3403
SOLVED! Episode 3403 Steps: 3 Epsilon 0.2988
Episode: 3404
Episode: 3405
SOLVED! Episode 3405 Steps: 1 Epsilon 0.2988
Episode: 3406
SOLVED! Episode 3406 Steps: 2 Epsilon 0.2988
Episode: 3407
Episode: 3408
SOLVED! Episode 3408 Steps: 3 Epsilon 0.2988
Episode: 3409
Episode: 3410
Episode: 3411
Episode: 3412
Episode: 3413
SOLVED! Episode 3413 Steps: 10 Epsilon 0.2988
Episode: 3414
SOLVED! Episode 3414 Steps: 1 Epsilon 0.2988
Episode: 3415
SOLVED! Episode 3415 Steps: 2 Epsilon 0.2988
Episode: 3416
Episode: 3417
Episode: 3418
Episode: 3419
Episode: 3420
Episode: 3421
Episode: 3422
Episode: 3423
Episode: 3424
Episode: 3425
SOLVED! Episode 3425 Steps: 2 Epsilon 0.2988
Episode: 3426
SOLVED! Episode 3426 Steps: 3 Epsilon 0.2988
Epi


******************************
Deterministic
******************************
Solved: 46
defaultdict(<class 'int'>, {3: 20, 1: 17, 2: 9})
******************************
Episode: 3500
Episode: 3501
Episode: 3502
Episode: 3503
SOLVED! Episode 3503 Steps: 3 Epsilon 0.2988
Episode: 3504
Episode: 3505
SOLVED! Episode 3505 Steps: 3 Epsilon 0.2988
Episode: 3506
SOLVED! Episode 3506 Steps: 3 Epsilon 0.2988
Episode: 3507
SOLVED! Episode 3507 Steps: 2 Epsilon 0.2988
Episode: 3508
Episode: 3509
Episode: 3510
Episode: 3511
SOLVED! Episode 3511 Steps: 3 Epsilon 0.2988
Episode: 3512
Episode: 3513
SOLVED! Episode 3513 Steps: 3 Epsilon 0.2988
Episode: 3514
Episode: 3515
Episode: 3516
SOLVED! Episode 3516 Steps: 2 Epsilon 0.2988
Episode: 3517
Episode: 3518
Episode: 3519
SOLVED! Episode 3519 Steps: 3 Epsilon 0.2988
Episode: 3520
SOLVED! Episode 3520 Steps: 21 Epsilon 0.2988
Episode: 3521
Episode: 3522
SOLVED! Episode 3522 Steps: 3 Epsilon 0.2988
Episode: 3523
SOLVED! Episode 3523 Steps: 3 Epsilon 0.2988



******************************
Deterministic
******************************
Solved: 44
defaultdict(<class 'int'>, {3: 22, 1: 15, 2: 7})
******************************
Episode: 3600
Episode: 3601
Episode: 3602
SOLVED! Episode 3602 Steps: 1 Epsilon 0.2988
Episode: 3603
Episode: 3604
Episode: 3605
Episode: 3606
SOLVED! Episode 3606 Steps: 2 Epsilon 0.2988
Episode: 3607
SOLVED! Episode 3607 Steps: 2 Epsilon 0.2988
Episode: 3608
Episode: 3609
Episode: 3610
SOLVED! Episode 3610 Steps: 3 Epsilon 0.2988
Episode: 3611
Episode: 3612
SOLVED! Episode 3612 Steps: 3 Epsilon 0.2988
Episode: 3613
SOLVED! Episode 3613 Steps: 3 Epsilon 0.2988
Episode: 3614
SOLVED! Episode 3614 Steps: 1 Epsilon 0.2988
Episode: 3615
Episode: 3616
Episode: 3617
SOLVED! Episode 3617 Steps: 2 Epsilon 0.2988
Episode: 3618
SOLVED! Episode 3618 Steps: 3 Epsilon 0.2988
Episode: 3619
Episode: 3620
Episode: 3621
Episode: 3622
Episode: 3623
Episode: 3624
Episode: 3625
SOLVED! Episode 3625 Steps: 15 Epsilon 0.2988
Episode: 3626
SOL